## 건설기계 오일 상태 분류 AI 경진대회

In [3]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import * 
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

In [4]:
data=pd.read_csv("train.csv")

In [5]:
data.head()

,ID,COMPONENT_ARBITRARY,ANONYMOUS_1,YEAR,SAMPLE_TRANSFER_DAY,ANONYMOUS_2,AG,AL,B,BA,...,U25,U20,U14,U6,U4,V,V100,V40,ZN,Y_LABEL
0,TRAIN_00000,COMPONENT3,1486,2011,7,200,0,3,93,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,154.0,75,0
1,TRAIN_00001,COMPONENT2,1350,2021,51,375,0,2,19,0,...,2.0,4.0,6.0,216.0,1454.0,0,NaN,44.0,652,0
2,TRAIN_00002,COMPONENT2,2415,2015,2,200,0,110,1,1,...,0.0,3.0,39.0,11261.0,41081.0,0,NaN,72.6,412,1
3,TRAIN_00003,COMPONENT3,7389,2010,2,200,0,8,3,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.3,7,0
4,TRAIN_00004,COMPONENT3,3954,2015,4,200,0,1,157,0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,133.1,128,0


In [6]:
data.isna().sum()

ID                         0
COMPONENT_ARBITRARY        0
ANONYMOUS_1                0
YEAR                       0
SAMPLE_TRANSFER_DAY        0
ANONYMOUS_2                0
AG                         0
AL                         0
B                          0
BA                         0
BE                         0
CA                         0
CD                      1394
CO                         0
CR                         0
CU                         0
FH2O                   10205
FNOX                   10205
FOPTIMETHGLY           10205
FOXID                  10205
FSO4                   10205
FTBN                   10205
FE                         0
FUEL                   10205
H2O                        0
K                       2299
LI                         0
MG                         0
MN                         0
MO                         0
NA                         0
NI                         0
P                          0
PB                         0
PQINDEX       

In [11]:
col.append('Y_LABEL')

In [12]:
data=data[col]

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14095 entries, 0 to 14094
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   14095 non-null  object 
 1   COMPONENT_ARBITRARY  14095 non-null  object 
 2   ANONYMOUS_1          14095 non-null  int64  
 3   YEAR                 14095 non-null  int64  
 4   ANONYMOUS_2          14095 non-null  int64  
 5   AG                   14095 non-null  int64  
 6   CO                   14095 non-null  int64  
 7   CR                   14095 non-null  int64  
 8   CU                   14095 non-null  int64  
 9   FE                   14095 non-null  int64  
 10  H2O                  14095 non-null  float64
 11  MN                   14095 non-null  int64  
 12  MO                   14095 non-null  int64  
 13  NI                   14095 non-null  int64  
 14  PQINDEX              14095 non-null  int64  
 15  TI                   14095 non-null 

In [14]:
data.fillna(0,inplace=True)

In [15]:
data.corr()

,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN,Y_LABEL
ANONYMOUS_1,1.000000,0.106546,0.072154,-0.025674,-0.003783,-0.007035,-0.014086,0.000440,0.003868,-0.004212,-0.006206,-0.007862,0.002277,0.001832,0.001883,0.019542,-0.019991,0.044197
YEAR,0.106546,1.000000,0.138199,-0.129124,-0.052337,-0.028871,-0.137593,-0.057709,0.009855,-0.039132,-0.091482,-0.049206,-0.067160,0.006399,-0.027624,-0.051584,0.048572,-0.087871
ANONYMOUS_2,0.072154,0.138199,1.000000,-0.006266,-0.000070,-0.001826,-0.002244,-0.004608,-0.003760,0.003304,-0.008465,-0.003914,-0.010409,-0.000981,-0.006469,-0.025339,0.032758,-0.033641
AG,-0.025674,-0.129124,-0.006266,1.000000,0.008861,0.005397,0.051046,0.025959,-0.003850,0.019507,0.013124,0.054014,0.030929,0.003609,-0.003853,0.013592,0.002631,0.014671
CO,-0.003783,-0.052337,-0.000070,0.008861,1.000000,0.173352,0.265347,0.536765,0.059836,0.425149,0.001389,0.402024,0.174601,0.263407,0.271112,0.107868,-0.018264,0.008175
CR,-0.007035,-0.028871,-0.001826,0.005397,0.173352,1.000000,0.050596,0.314226,0.037587,0.247603,-0.008194,0.221551,0.147009,0.226221,0.391048,0.079090,-0.053180,0.014233
CU,-0.014086,-0.137593,-0.002244,0.051046,0.265347,0.050596,1.000000,0.287277,-0.001719,0.222300,-0.017165,0.486860,0.120772,0.102578,0.089311,-0.067575,0.040869,0.024975
FE,0.000440,-0.057709,-0.004608,0.025959,0.536765,0.314226,0.287277,1.000000,0.118843,0.622415,-0.063966,0.583322,0.427582,0.384018,0.341445,0.247786,-0.155119,0.047992
H2O,0.003868,0.009855,-0.003760,-0.003850,0.059836,0.037587,-0.001719,0.118843,1.000000,0.144325,-0.009545,0.051752,0.081923,0.153990,0.048667,0.371215,-0.031378,-0.004262
MN,-0.004212,-0.039132,0.003304,0.019507,0.425149,0.247603,0.222300,0.622415,0.144325,1.000000,-0.047064,0.527618,0.376253,0.585042,0.375177,0.149613,-0.043602,0.024274


In [16]:
x=data.drop(['Y_LABEL','COMPONENT_ARBITRARY','ID'],axis=1)

In [17]:
y=data['Y_LABEL']

In [106]:
x=pd.get_dummies(x,'COMPONENT_ARBITRARY')

In [18]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 2022)

#### XGB

In [132]:
from xgboost import XGBClassifier

In [140]:
xgb=XGBClassifier()
xgb.fit(x_train,y_train)
xgb_pred=xgb.predict(x_val)
print(classification_report(y_val,xgb_pred))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95      3850
           1       0.41      0.04      0.08       379

    accuracy                           0.91      4229
   macro avg       0.66      0.52      0.52      4229
weighted avg       0.87      0.91      0.87      4229



#### RF

In [1]:
from sklearn.ensemble import RandomForestClassifier

In [19]:
rf=RandomForestClassifier()
rf.fit(x_train,y_train)

RandomForestClassifier()

In [20]:
rf_pred=rf.predict(x_val)

In [21]:
print(classification_report(y_val,rf_pred))

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      3850
           1       0.69      0.02      0.05       379

    accuracy                           0.91      4229
   macro avg       0.80      0.51      0.50      4229
weighted avg       0.89      0.91      0.87      4229



#### test

In [30]:
test_data=pd.read_csv("test.csv")

In [10]:
col=list(test_data.columns)

In [22]:
test_data.fillna(0,inplace=True)
test_data.drop(['COMPONENT_ARBITRARY','ID'],axis=1,inplace=True)
#test_data=pd.get_dummies(test_data,'COMPONENT_ARBITRARY')

In [23]:
test_data

,ANONYMOUS_1,YEAR,ANONYMOUS_2,AG,CO,CR,CU,FE,H2O,MN,MO,NI,PQINDEX,TI,V,V40,ZN
0,2192,2016,200,0,0,0,1,12,0.0,0,0,0,10,0,0,91.3,1091
1,2794,2011,200,0,0,2,1,278,0.0,3,0,0,2732,1,0,126.9,12
2,1982,2010,200,0,0,0,16,5,0.0,0,0,0,11,0,0,44.3,714
3,1404,2009,200,0,0,3,4,163,0.0,4,3,0,8007,0,0,142.8,94
4,8225,2013,200,0,0,0,6,13,0.0,0,0,0,16,0,0,63.4,469
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,1714,2014,200,0,0,3,130,1047,0.0,65,5,2,1999,0,0,71.2,1163
6037,4131,2016,200,0,0,5,2,736,0.0,5,0,0,693,0,0,137.8,13
6038,4325,2014,200,0,0,0,0,53,0.0,0,0,0,69,0,0,302.2,13
6039,1364,2013,200,0,0,0,62,2,0.0,0,0,0,5,0,0,51.1,478


In [25]:
test_xgb_pred=rf.predict(test_data)

In [26]:
df=pd.DataFrame(test_xgb_pred)

In [29]:
df

,0
0,0
1,0
2,0
3,0
4,0
...,...
6036,0
6037,0
6038,0
6039,0


In [31]:
ID=pd.DataFrame(test_data.ID)

In [32]:
result=pd.concat([ID,df],axis=1)

In [33]:
result

,ID,0
0,TEST_0000,0
1,TEST_0001,0
2,TEST_0002,0
3,TEST_0003,0
4,TEST_0004,0
...,...,...
6036,TEST_6036,0
6037,TEST_6037,0
6038,TEST_6038,0
6039,TEST_6039,0


In [34]:
result.rename(columns={0:'Y_LABEL'},inplace=True)

In [35]:
result.to_csv("result.csv",index=False)